# EMOD Pseudocode

_Indented_ code under commented out functions is the pseudocode for the function. E.g., `updateInfectivity(dt)` calls `computeMaxInfectionProb(dt)` and `gap = calcGap()`.

```cpp
void Node::Update(float dt) {
    ...
    // updateInfectivity(dt);
        // computeMaxInfectionProb( dt );
            ProbabilityNumber prob = EXPCDF(-contagion * dt);
            gap = 1;           // default to visiting every individual
            bSkipping = false; // default to visiting (!skipping) every individual
        // gap = calcGap();
            int gap = 1;
            float maxProb = prob;
            if (maxProb >= 1.0) {
                gap = 1;
            }
            else if (maxProb > 0.0) {
                // geometric based on maxProb
                gap = int(ceil(log(1.0 - GetRng()->e()) / log(1.0 - maxProb)));	

    ...
    for (int i = 0; i < individualHumans.size(); ++i ) {
    	// PreUpdate();
            if (gap == 1) {
                bSkipping = false;  // visit this individual
                gap = calcGap();    // update gap (see above)
            }
            else {
                bSkipping = true;   // skip this individual
                --gap;              // decrement gap counter
            }
        // individualHumans[i]->Update(...);
            // ExposeToInfectivity(...);
                // parent->ExposeIndividual(...);
                    if (!bSkipping) {
                        // transmissionGroups->ExposeToContagion(...);
                            // candidate->Expose(...);
                                auto indProb = -cp->GetTotalContagion() * dt;
                                indProb *= susceptibility->getModAcquire();
                                indProb *= susceptibility->getModRisk();
                                indProb *= interventions->GetInterventionReducedAcquire(...);
                                indProb = EXPCDF(indProb);
                                bool acquire = false;
                                float maxProb = parent->GetMaxInfectionProb(...);
                                if (maxProb > 0) {
                                    // if individual is maximally susceptible
                                    if (maxProb == prob) {
                                        acquire = true;
                                    }
                                    // if individual is _not_ maximally susceptible
                                    else if (prob/maxProb > GetRng()->e()) {
                                        acquire = true;
                                    }
                                }
                    }
    }
    ...
}

#define EXPCDF(x)   (1 - exp(x))
```

In [ ]:
# might be necessay on a fresh Codespace
%pip install numba

In [1]:
import numpy as np
import numba as nb

In [3]:
@nb.njit()
def skip(probability: np.float32) -> np.uint32:
    n = np.uint32(np.ceil(np.log(1.0-np.random.rand())/np.log(1.0-probability))) if probability < 1.0 else np.uint32(1)
    return n

In [27]:
@nb.njit()
def visit(susceptibility: np.ndarray, probability: np.float32) -> np.uint32:
    visited = 0
    index = skip(probability) - 1
    while (index >= 0) and (index < len(susceptibility)):
        if susceptibility[index] == 1.0:
            visited += 1
        elif np.random.rand() < susceptibility[index]:
            visited += 1
        index += skip(probability)
    return visited

In [64]:
susceptibility = np.ones(1_000_000, dtype=np.float32)

In [65]:
# randomize susceptibility uniformly between 0 and 1
susceptibility *= np.random.uniform(0.0, 1.0, size=susceptibility.shape)

In [68]:
# show a few samples of susceptibility
np.random.choice(susceptibility,32, replace=False)

array([0.4846219 , 0.7550575 , 0.26625404, 0.5665095 , 0.5553552 ,
       0.76844317, 0.62275714, 0.01168378, 0.52227485, 0.95538485,
       0.88241625, 0.3677364 , 0.8454586 , 0.41678628, 0.12353604,
       0.358242  , 0.68609846, 0.4122491 , 0.53500694, 0.8286768 ,
       0.5153037 , 0.6476279 , 0.60792714, 0.68498   , 0.1507953 ,
       0.06161552, 0.649481  , 0.92346513, 0.37631765, 0.32190898,
       0.69754404, 0.6521351 ], dtype=float32)

In [76]:
for contagion in [0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0]:
    probability = contagion / susceptibility.shape[0]
    tries = 100_000
    visits = np.zeros(tries, dtype=np.uint32)
    for i in range(tries):
        visits[i] = visit(susceptibility, probability)
    print(f"Contagion: {contagion:>8.3f}, Probability: {probability:>11.9f}, Visits: {visits.mean():>7.3f} ± {visits.std():5.2f}    ", end="")
    print(f"mean sus.: {susceptibility.mean():3.2}, eff. prob.: {(np.float64(probability)*susceptibility.mean()):>11.9f}")

Contagion:    0.125, Probability: 0.000000125, Visits:   0.062 ±  0.25    mean sus.: 0.5, eff. prob.: 0.000000063
Contagion:    0.250, Probability: 0.000000250, Visits:   0.124 ±  0.35    mean sus.: 0.5, eff. prob.: 0.000000125
Contagion:    0.500, Probability: 0.000000500, Visits:   0.247 ±  0.50    mean sus.: 0.5, eff. prob.: 0.000000250
Contagion:    1.000, Probability: 0.000001000, Visits:   0.499 ±  0.71    mean sus.: 0.5, eff. prob.: 0.000000500
Contagion:    2.000, Probability: 0.000002000, Visits:   0.999 ±  1.00    mean sus.: 0.5, eff. prob.: 0.000001000
Contagion:    4.000, Probability: 0.000004000, Visits:   2.000 ±  1.42    mean sus.: 0.5, eff. prob.: 0.000002001
Contagion:    8.000, Probability: 0.000008000, Visits:   3.997 ±  2.00    mean sus.: 0.5, eff. prob.: 0.000004001
Contagion:   16.000, Probability: 0.000016000, Visits:   7.999 ±  2.82    mean sus.: 0.5, eff. prob.: 0.000008002
Contagion:   32.000, Probability: 0.000032000, Visits:  16.017 ±  4.01    mean sus.: 0.5